### pc2beam demo
This notebook demonstratest the basic functionality of the pc2beam package.

setup

In [2]:
import sys
from pathlib import Path
# add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from pc2beam.data import PointCloud

/Users/fnoi/Code/pc2beam/venv_pc2beam/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


load data and visualize input

compute $\vec{s_1}$ and evaluate

compute $\vec{s_2}$ and evaluate